# Gradient Boosting Machine Regressor 

## Mục tiêu
- Xây dựng mô hình Gradient Boosting Regressor để dự đoán tuổi thọ trung bình.

- Sử dụng dữ liệu đã được tiền xử lý từ `data/processed/`.

- Tối ưu hóa siêu tham số bằng 5-Fold Cross-Validation.

- Do MAE đo sai số theo đúng đơn vị thực tế (năm tuổi thọ). Ta sử dụng tiêu chuẩn là độ đo MAE để chọn siêu tham số cuối cùng.

- Đánh giá mô hình trên tập train.

## Giới thiệu
Gradient Boosting Machine (GBM) là một thuật toán ensemble learning mạnh mẽ, xây dựng mô hình theo cách tuần tự (sequential) thay vì song song như Random Forest. Mỗi cây mới được xây dựng để sửa chữa lỗi của các cây trước đó.

### **Ưu điểm:**
- Hiệu suất dự đoán rất cao, thường vượt trội Random Forest.

- Xử lý tốt các mối quan hệ phi tuyến phức tạp.

- Tự động thực hiện chọn lọc đặc trưng.

### **Nhược điểm:**
- Dễ bị overfitting.

- Thời gian huấn luyện lâu.

- Nhạy cảm với siêu tham số.

- Khó giải thích hơn các mô hình đơn giản.


## Bước 1 - Import các thư viện cần thiết

In [19]:
# Thư viện cơ bản
import numpy as np
import pandas as pd
import os

# Thư viện sklearn cho mô hình và đánh giá
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Thư viện để lưu mô hình
import joblib

## Bước 2 - Đọc dữ liệu đã tiền xử lý

Đọc dữ liệu từ các file CSV đã được tiền xử lý

In [20]:
# Đọc dữ liệu
train_df = pd.read_csv('../data/processed/train.csv')

print("THÔNG TIN DỮ LIỆU")
print("="*60)
print(f"Kích thước tập train: {train_df.shape}")

# Hiển thị 5 dòng đầu của tập train
print("\n5 dòng đầu tiên của tập train (đã được chuẩn hóa):")
train_df.head()

THÔNG TIN DỮ LIỆU
Kích thước tập train: (3124, 13)

5 dòng đầu tiên của tập train (đã được chuẩn hóa):


,country_name,country_code,year,population,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,labor_force
0,Denmark,DNK,2017,-0.203264,-0.418953,81.102439,1.682694,-0.047741,1.673510,0.642797,0.754689,0.118599,0.069223
1,"Korea, Dem. People's Rep.",PRK,2017,-0.056076,-0.530921,73.034000,-0.412271,0.028651,-0.245951,-1.294730,0.439985,-0.216848,2.087276
2,Madagascar,MDG,2008,-0.091998,1.007790,61.992000,-0.607170,0.547296,-1.651029,-2.194047,-2.540432,-0.528445,2.542268
3,Greece,GRC,2018,-0.166799,-0.945727,81.787805,0.170491,-0.209157,1.356075,0.642797,0.754689,0.161554,-1.033944
4,South Sudan,SSD,2019,-0.169071,1.000983,58.129000,-0.412271,0.028651,-1.375450,-2.593358,-2.691194,-0.278341,1.279234


## Bước 3 - Chuẩn bị dữ liệu cho mô hình

Tách biến mục tiêu (`life_expectancy`) khỏi các đặc trưng. Loại bỏ các cột không cần thiết như `country_name`, `country_code`.

In [21]:
# Định nghĩa các cột dùng để dự đoán
feature_cols = [col for col in train_df.columns 
                if col not in ['life_expectancy', 'country_name', 'country_code']]

# Tách X và y cho từng tập
X_train = train_df[feature_cols]
y_train = train_df['life_expectancy']

print("THÔNG TIN CÁC TẬP DỮ LIỆU")
print("="*60)
print(f"Số lượng đặc trưng: {len(feature_cols)}")
print(f"\nCác đặc trưng được sử dụng:")
for i, col in enumerate(feature_cols, 1):
    print(f"  {i}. {col}")

print(f"\nKích thước X_train: {X_train.shape}")
print(f"Kích thước y_train: {y_train.shape}")

THÔNG TIN CÁC TẬP DỮ LIỆU
Số lượng đặc trưng: 10

Các đặc trưng được sử dụng:
  1. year
  2. population
  3. pop_growth
  4. gdp_per_capita
  5. gdp_growth
  6. sanitation
  7. electricity
  8. water_access
  9. co2_emissions
  10. labor_force

Kích thước X_train: (3124, 10)
Kích thước y_train: (3124,)


## Bước 4 - Xây dựng và huấn luyện mô hình GBM

### 4.1. Sử dụng RandomizedSearchCV với 5-Fold Cross-Validation để tìm kiếm siêu tham số tối ưu cho mô hình Gradient Boosting.

In [22]:
from scipy.stats import randint, uniform

# Khai báo lưới siêu tham số cần tìm
param_distributions = {
    "n_estimators": randint(100, 600),        
    "learning_rate": uniform(0.01, 0.19),     
    "max_depth": randint(2, 7),               
    "min_samples_split": randint(2, 21),      
    "min_samples_leaf": randint(1, 11),       
    "subsample": uniform(0.6, 0.4),           
    "max_features": ["sqrt", "log2", None],
}

gbm = GradientBoostingRegressor(random_state=42)

random_search = RandomizedSearchCV(
    estimator=gbm,
    param_distributions=param_distributions,
    n_iter=60,               
    cv=5,                  
    scoring="neg_mean_absolute_error", 
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Huấn luyện mô hình
random_search.fit(X_train, y_train)

Fitting 5 folds for each of 60 candidates, totalling 300 fits


,estimator,GradientBoost...ndom_state=42)
,param_distributions,"{'learning_rate': <scipy.stats....0020DFCDF9F10>, 'max_depth': <scipy.stats....0020DFCDFA350>, 'max_features': ['sqrt', 'log2', ...], 'min_samples_leaf': <scipy.stats....0020DFCDFB490>, ...}"
,n_iter,60
,scoring,'neg_mean_absolute_error'
,n_jobs,-1
,refit,True
,cv,5
,verbose,2
,pre_dispatch,'2*n_jobs'
,random_state,42
,error_score,nan


### 4.2. Hiển thị siêu tham số tối ưu và đánh giá hiệu suất của mô hình tốt nhất

In [23]:
# Lấy mô hình tốt nhất và tham số tối ưu
best_gbm = random_search.best_estimator_
best_params = random_search.best_params_

print("SIÊU THAM SỐ TỐI ƯU")
print("="*60)

for param, value in best_params.items():
    print(f"  {param:20s}: {value}")

# Thông tin về mô hình tối ưu
print(f"\nSố lượng cây: {best_gbm.n_estimators}")
print(f"Learning rate: {best_gbm.learning_rate}")
print(f"Max depth: {best_gbm.max_depth}")
print(f"Subsample: {best_gbm.subsample}")

SIÊU THAM SỐ TỐI ƯU
  learning_rate       : 0.1309031974553201
  max_depth           : 6
  max_features        : None
  min_samples_leaf    : 6
  min_samples_split   : 9
  n_estimators        : 382
  subsample           : 0.7737577462041715

Số lượng cây: 382
Learning rate: 0.1309031974553201
Max depth: 6
Subsample: 0.7737577462041715


### 4.3. In ra CV MAE loss

In [24]:
print("KẾT QUẢ MÔ HÌNH TỐI ƯU")
print("="*60)
print("Model: GBM:")
print(f"Cross-validation MAE loss: {-random_search.best_score_}")


KẾT QUẢ MÔ HÌNH TỐI ƯU
Model: GBM:
Cross-validation MAE loss: 1.1486264981386776


### 4.4. Lưu mô hình

In [25]:
import joblib
import os

# Tạo thư mục lưu model nếu chưa có
os.makedirs('../models/5_GBM', exist_ok=True)

# Lưu mô hình
joblib.dump(best_gbm, "../models/5_GBM/GBM.pkl.gz", compress=('gzip', 6))

['../models/5_GBM/GBM.pkl.gz']

## Kết luận

### Tổng kết:
- Mô hình Gradient Boosting Machine đã được xây dựng và tối ưu hóa thành công.

- RandomizedSearchCV với 5-Fold CV đã tìm được bộ siêu tham số tối ưu.